<a href="https://youtu.be/A2euuevpYis" target="_parent"><img src="https://upload.wikimedia.org/wikipedia/commons/thumb/0/09/YouTube_full-color_icon_%282017%29.svg/71px-YouTube_full-color_icon_%282017%29.svg.png" alt="Open In Colab"/></a>


# **Práctico Sistemas Recomendadores: pyreclab - Slope One**

En este práctico seguiremos utilizando [pyreclab](https://github.com/gasevi/pyreclab), con el cual estamos aprendiendo distintas técnicas de recomendación. Seguiremos usando la misma base de datos de los prácticos anteriores, para que puedan comparar los métodos y sus implementaciones. Este práctico está acompañado de un [video comentando la actividad](https://youtu.be/A2euuevpYis).

En esta oportunidad exploraremos el recomendador de Pendiente Uno o **Slope One** [1].

**Adaptado y preparado por:** Francisca Cattan 📩 fpcattan@uc.cl

Referencias 📖
------
[1] *Lemire, D., & Maclachlan, A. (2005, April). Slope One Predictors for Online Rating-Based Collaborative Filtering. In SDM (Vol. 5, pp. 1-5).*


**Nombre**:  Sergio Gazali Allen


## Actividad 1 👓

Antes de empezar con el práctico, responde la siguiente pregunta con lo visto en clases.

**Pregunta:** Explique cómo funciona Slope One (como modelo teórico, no piense en la implementación). En particular explique:

- Repasemos: ¿Por qué este recomendador es un algoritmo de Filtrado Colaborativo?
- Este Filtrado Colaborativo, ¿está basado en el usuario o en los items? ¿Por qué?
- ¿Qué datos recibe Slope One y qué hace con ellos? (qué tipo de columnas y qué calculo)
- ¿Qué pasaría si se agrega un nuevo rating a la base de datos?
- Opcional: ¿Cómo crees que le iría al recomendador con un usuario que acaba de entrar al sistema y ha asignado muy pocos ratings?

💡 *Hint: La bibliografía todo lo puede.*

**Respuesta:**

- El algoritmo lo que hace es predecir ratings de usuarios a ítems en base a ratings de otros usuarios a dichos ítems (y a otros ítems para establecer las relaciones), para luego recomendarlos. Es decir, filtra o evalúa según las opiniones (ratings) de otras personas, lo que lo hace un algoritmo de filtrado colaborativo.

- Está basado en los ítems. Porque lo que hace es predecir el rating de un ítem por un usuario, y la manera de hacer esto es basada en ítems, ya que toma pares de ítems y según las diferencias en sus ratings promedios estima cómo debiera variar el rating desconocido del usuario respecto a los ratings que ha dado a los otros ítems.

- Recibe datos de usuarios, ítems y los respectivos ratings que los primeros le dan a los segundos, es decir tres columnas o tipos de datos escenciales: id_usuario, id_item, rating. Con ellos calcula desviaciones promedio en los ratings de pares de ítems, para luego aplicar estas desviaciones, ponderándolas por cantidad de pares de ratings entre dichos ítems, sobre los ratings conocidos del usuario a predecir, para obtener un valor estimado de su rating para un ítem. 

- Se pueden actualizar las predicciones instantáneamente. La desviación promedio entre dos ítems es una sumatoria de las diferencias de rating dados para ellos por cada usuario. Si un usuario ingresa un nuevo rating que permita actualizar este dato, basta con actualizar este valor y se podrán tener nuevas predicciones.

- No tendrá problema al estimar el rating basado únicamente en cada ítem que el usuario ya calificó, porque esto sólo necesita la desviación promedio de los ítems y el rating asignado por el usuario. El problema es que al hacer la poderación de esas predicciones para obtener una predicción final, estará ponderando muy pocas predicciones, lo que podemos concluir que será poco informativo de los gustos generales del usuario. 




# **Configuración Inicial**

## Paso 1:
Descargue directamente a Colab los archivos del dataset ejecutando las siguientes 3 celdas:


In [2]:
!curl -L -o "u1.base" "https://drive.google.com/uc?export=download&id=1bGweNw7NbOHoJz11v6ld7ymLR8MLvBsA"

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   388    0   388    0     0   1182      0 --:--:-- --:--:-- --:--:--  1182
100 1546k  100 1546k    0     0  2648k      0 --:--:-- --:--:-- --:--:-- 2648k


In [3]:
!curl -L -o "u1.test" "https://drive.google.com/uc?export=download&id=1f_HwJWC_1HFzgAjKAWKwkuxgjkhkXrVg"

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   388    0   388    0     0    497      0 --:--:-- --:--:-- --:--:--   496
100  385k  100  385k    0     0   343k      0  0:00:01  0:00:01 --:--:--  188M


In [4]:
!curl -L -o "u.item" "https://drive.google.com/uc?export=download&id=10YLhxkO2-M_flQtyo9OYV4nT9IvSESuz"

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   388    0   388    0     0    586      0 --:--:-- --:--:-- --:--:--   586
100  230k  100  230k    0     0   225k      0  0:00:01  0:00:01 --:--:-- 1479k


Los archivos **u1.base** y **u1.test** tienen tuplas {usuario, item, rating, timestamp}, que es la información de preferencias de usuarios sobre películas en una muestra del dataset [movielens](https://grouplens.org/datasets/movielens/).

## Paso 2:

Instalamos pyreclab utilizando pip.

In [5]:
!pip install pyreclab --upgrade

     |████████████████████████████████| 234 kB 14.6 MB/s 


## Paso 3:

Hacemos los imports necesarios para este práctico.

In [6]:
import pyreclab
import numpy as np
import pandas as pd

# **El dataset**

💡 *En prácticos anteriores, vimos como analizar este dataset. Puedes revisarlos en caso de dudas.*

## Paso 4:

Ya que queremos crear una lista de recomendación de items para un usuario en especifico, necesitamos obtener información adicional de cada película tal como título, fecha de lanzamiento, género, etc. Cargaremos el archivo de items descargado "u.item" para poder mapear cada identificador de ítem al conjunto de datos que lo describe.

In [7]:
# Definimos el orden de las columnas
info_cols = [ 'movieid', 'title', 'release_date', 'video_release_date', 'IMDb_URL', \
              'unknown', 'Action', 'Adventure', 'Animation', 'Children', 'Comedy', \
              'Crime', 'Documentary', 'Drama', 'Fantasy', 'Film-Noir', 'Horror', \
              'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western' ]

# Asignamos a una variable la estructura de datos de los items
info_file = pd.read_csv('u.item', sep='|', index_col = 0, names = info_cols, header=None, encoding='latin-1')

# **Slope One**

## Paso 5:

Seguiremos un camino muy similar a los ejercicios de User KNN e Item KNN. Crearemos una instancia del algoritmo de recomendación y luego pasaremos a la fase de entrenamiento.

In [8]:
# Declaramos la instancia SlopeOne
mySlopeOne = pyreclab.SlopeOne(dataset='u1.base', dlmchar=b'\t', header=False, usercol=0, itemcol=1, ratingcol=2)

In [9]:
# Y enntrenamos
mySlopeOne.train()

## Actividad 2 👓

**Pregunta:** Explique qué hace el método `train()` en este caso, dado el modelo teórico. ¿Calcula información?, ¿no hace nada?, ¿ordena los datos? 

**Respuesta:**

Sí calcula información. Específicamente calcula una matriz cuyas filas y columnas representan ítems, y cuyos elementos contienen las desviaciones promedio entre cada par de ítems (elemento (i, j) contiene la desviación promedio entre los ítems i y j).

## Paso 6:

Llego la hora de predecir el rating.

In [10]:
# Esta es la predicción de rating que el usuario ID:457 otorgaría al ítem ID:37
# De esta forma podemos comparar el resultado con los prácticos anteriores
mySlopeOne.predict("457", "37")

3.2408759593963623

In [11]:
# También podemos guardar la predicción en una variable
prediction = mySlopeOne.predict("457", "37")

In [12]:
# Podemos comprobar las peliculas rankeadas por el usuario ID:457
# Que ciertamente ha participado activamente (¡156 items!)
train_file = pd.read_csv('u1.base', sep='\t', names = ['userid', 'itemid', 'rating', 'timestamp'], header=None)
train_file[train_file['userid'] == 457]

,userid,itemid,rating,timestamp
37269,457,1,4,882393244
37270,457,7,4,882393278
37271,457,9,5,882393485
37272,457,11,4,882397020
37273,457,13,3,882393883
...,...,...,...,...
37420,457,1047,2,882395964
37421,457,1119,4,882398308
37422,457,1168,5,882548761
37423,457,1210,4,882549905


In [13]:
# Y también cuáles usuarios han rankeado la pelicula ID:37
train_file[train_file['itemid'] == 37]

,userid,itemid,rating,timestamp
1302,13,37,1,882397011
14851,201,37,2,884114635
19670,268,37,3,876514002
29489,363,37,2,891498510
31084,385,37,4,880013483
32996,405,37,1,885548384
62777,773,37,3,888540352


## Actividad 3 👓

Haremos un pequeño experimento para entender mejor como funciona Slope One. Gracias al ejercicio anterior, sabemos que el usuario 457 ya ha asignado el mejor rating (5 ⭐) a las dos peliculas ID:9 e ID:1168. Comparemos.

**Pregunta:** ¿Cómo se explican estos resultados?  

**Respuesta:** Se ve que la predicción es relativamente buena, ya que efectivamente le asigna a estas dos películas un ranking bastante favorable o positivo, sin embargo hace una predicción menor al valor real. Me parece esperable que sea defícil predecir correctamente un ranking de 5, ya que este puntaje es un caso "límite" y es razonable que en un algoritmo que se basa en los rankings promedio de los ítems ocurra poco que obtengamos resultados en los límites. 

Mi manera de explicarlo sería: esta predicción se consigue en base a la desviación promedio entre el ítem en cuestión y otros ítems, junto con los ratings del usuario en cuestión a esos otros ítems. La manera en que estos valores pueden hacer que la predicción se acerque al 5 es que para algún par de películas A, B en que A tiene menor rating promedio, el rating del usuario para A sea suficientemente mayor a su rating promedio para que, al sumar la desviación promedio, se obtenga un valor mayor o igual a 5. Sin embargo, podríamos esperar que en muchos casos el usuario tenga un rating similar al rating promedio para A, en cuyo caso, sumar la desviación promedio resultará en un valor menor o igual a 5 (probablemente menor ya que los ratings promedios no solo están entre 1 y 5, sino que es muy poco probable que estén en los extremos, ya que son promedios).

In [14]:
prediction_id9 = mySlopeOne.predict("457", "9")
prediction_id1168 = mySlopeOne.predict("457", "1168")

print('Prediction for ID:9 :', prediction_id9)
print('Prediction for ID:1168 :', prediction_id1168)

Prediction for ID:9 : 4.530702114105225
Prediction for ID:1168 : 4.166153907775879


## Paso 7:

Generaremos ahora una lista ordenada de las top-N recomendaciones, dado un usuario.



In [22]:
# Mediante el método recommend() genereremos una lista top-5 recomendaciones para el usuario ID:457
reclist_slopeone = mySlopeOne.recommend("457", 5)

# Y visualizaremos el resultado
print('Lista de items según ID:', reclist_slopeone)

Lista de items según ID: ['1592', '1589', '1656', '1431', '1653']


In [18]:
# Lo convertimos a numpy array
recmovies_slopeone = np.array(reclist_slopeone).astype(int)

# Utilizamos la estructura de datos de los items para encontrar los títulos recomendados
print('Lista de items por nombre:')
info_file.loc[recmovies_slopeone]['title']

Lista de items por nombre:


movieid
1674                                    Mamma Roma (1962)
1653    Entertaining Angels: The Dorothy Day Story (1996)
1064                                     Crossfire (1947)
1651                         Spanish Prisoner, The (1997)
1650                              Butcher Boy, The (1998)
Name: title, dtype: object

## Actividad 4 👩🏻‍💻

Genera una nueva recomendacion, modificando los hiperparametros de usuario y topN a tu elección.

**Pregunta:** ¿Ves una diferencia en la recomendación entre el nuevo usuario y el usuario ID:457?

**Respuesta:** Sí, el resultado de la recomendación es diferente. Esto se debe a que slope one sí es personalizado, a diferencia de most popular o avarage rating, ya que las desviaciones promedio entre ítems son comunes para toda la población, pero estas desviaciones se suman a los ratings particulares que ha dado cada usuario para generar una recomendación.

In [26]:
# Escribe el nuevo codigo aqui
reclist_slopeone = mySlopeOne.recommend("400", 7)
recmovies_slopeone = np.array(reclist_slopeone).astype(int)
info_file.loc[recmovies_slopeone]['title']

movieid
1674                                    Mamma Roma (1962)
1653    Entertaining Angels: The Dorothy Day Story (1996)
1064                                     Crossfire (1947)
1651                         Spanish Prisoner, The (1997)
1650                              Butcher Boy, The (1998)
1645                              Butcher Boy, The (1998)
1642                             Some Mother's Son (1996)
Name: title, dtype: object

## Actividad 5 👩🏻‍💻

Dado el usuario ID:44, cree dos listas de películas recomendadas; la primera utilizando el algoritmo Most Popular y la segunda utilizando el algoritmo Slope One.

**Pregunta:** Realice un analisis apreciativo de las similitudes y diferencias entre ambas recomendaciones.

**Respuesta:** No hay ninguna película en común entre las dos listas, lo que refleja el impacto de la elección de algoritmo para recomendar. Sobre most popular, se hace claro al hacer este ejercicio que la recomendación no sigue un patrón específico de gustos, por lo que la lista recomendada no será muy útil para el usuario ya que, si bien son títulos populares, no hay nada que los asocie entre ellos. 

Además, parece muy difícil que este algoritmo entregue ítems que el usuario no conozca, o que no hubiera conocido por otros medios. Esto se me hizo muy notorio al comparar las listas, ya que la de most popular no fue para nada novedosa, pero slope one me entregó títulos que no conocía. 

Por otra parte, me llamó la atención que la gran mayoría de las películas en ambos casos son de la década de los 90'. Esto simplemente puede ser un sesgo de la base de datos. Sin embargo, las únicas recomendaciones de otras décadas entregadas por Most Popular corrsponden a Star Wars. Slope One, en cambio recomendó dos películas distintas de décadas novedosas (40', 50'). Lo que, nuevamente, me parece una señal de que es capaz de recomendar ítems más novedosos al usuario y menos obvios/conocidos.

In [32]:
# Escribe el nuevo codigo aqui
most_popular = pyreclab.MostPopular(dataset='u1.base',
                   dlmchar=b'\t',
                   header=False,
                   usercol=0,
                   itemcol=1,
                   ratingcol=2)
most_popular.train()

user_id = 44
top_n = 7
print('Lista recomendada por Most Popular:')
ranking = [int(r) for r in most_popular.recommend(str(user_id), top_n, includeRated=False)]
recmovies_most_popular = np.array(ranking).astype(int)
print('{}\n'.format(info_file.loc[recmovies_most_popular]['title']))
#print('Recommendation for user {}: {}'.format(user_id, ranking))


print('Lista recomendada por Slope One:')

reclist_slopeone = mySlopeOne.recommend("44", 7)
recmovies_slopeone = np.array(reclist_slopeone).astype(int)
#print('Recommendation for user {}: {}'.format(user_id, reclist_slopeone))

info_file.loc[recmovies_slopeone]['title']

Lista recomendada por Most Popular:
movieid
50                  Star Wars (1977)
100                     Fargo (1996)
181        Return of the Jedi (1983)
286      English Patient, The (1996)
1                   Toy Story (1995)
121    Independence Day (ID4) (1996)
300             Air Force One (1997)
Name: title, dtype: object

Lista recomendada por Slope One:


movieid
1656                        Little City (1998)
1064                          Crossfire (1947)
1643                         Angel Baby (1995)
1642                  Some Mother's Son (1996)
1625                         Nightwatch (1997)
1599             Someone Else's America (1995)
1512    World of Apu, The (Apur Sansar) (1959)
Name: title, dtype: object